# 02 - Failing fits

This tutorial shows examples of fits that fail and how to identify and fix the issue or avoid it in the first place.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data and model

Let's use the same data and model as in the previous notebooks.


In [2]:
df = pd.DataFrame({
    "x": [9, 14, 21, 28, 42, 57, 63, 70, 79],
    "y": [8.93, 10.80, 18.59, 22.33, 39.35, 56.11, 61.73, 64.62, 67.08]
})

In [3]:
def model(x, b1, b2, b3):
    return b1 / (1 + np.exp(b2 - b3 * x))

## A failing fit

Fits and warnings and errors can sometimes change depending on your system and library versions. For me, the following fit gives two warnings:

1. RuntimeWarning: overflow encountered in exp
2. OptimizeWarning: Covariance of the parameters could not be estimated

In [4]:
import scipy.optimize

In [5]:
popt, pcov = scipy.optimize.curve_fit(f=model, xdata=df["x"], ydata=df["y"])

/Users/cdeil/anaconda3/envs/py-model-fit/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/6v/0_6nt0pj07x9xjhd8qzkyy700000gn/T/ipykernel_20821/1918929932.py:1: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = scipy.optimize.curve_fit(f=model, xdata=df["x"], ydata=df["y"])


In [6]:
popt

array([  38.83777778, -986.43411354,  111.60437184])

In [7]:
pcov

array([[inf, inf, inf],
       [inf, inf, inf],
       [inf, inf, inf]])

In this case our model and cost function only has one `exp` call `np.exp(b2 - b3 *x)` so it's clear that the overflow happened because `b2 - b3 * x` was too large.
Also the `pcov` being `inf` makes sense if the optimiser warns that covariance cound not be estimated.

But some `popt` was computed. Is it valid? 

The [scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) docstring says this:
> Raises: RuntimeError: if the least-squares minimization fails. 

Well, in this case `popt` is not valid. The fit failed and the function didn't raise a `RuntimeError`.

**Learning 1: if you get any warnings/errors, don't trust the fit. Fix them.**


## What's the problem?

In this case, and actually it's probably the most common issue of failing fits, the problem is that the initial guess for the parameters `p0` was bad.

As stated in the [scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) docstring description of `p0`:
> Initial guess for the parameters (length N). If None, then the initial values will all be 1

So effectively we were doing this:

In [8]:
p0=[1, 1, 1]
popt, pcov, infodict, msg, ier = scipy.optimize.curve_fit(f=model, xdata=df["x"], ydata=df["y"], p0=p0, full_output=True)

/Users/cdeil/anaconda3/envs/py-model-fit/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/6v/0_6nt0pj07x9xjhd8qzkyy700000gn/T/ipykernel_20821/3188724692.py:2: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov, infodict, msg, ier = scipy.optimize.curve_fit(f=model, xdata=df["x"], ydata=df["y"], p0=p0, full_output=True)


In [9]:
infodict

{'fvec': array([ 29.90777778,  28.03777778,  20.24777778,  16.50777778,
         -0.51222222, -17.27222222, -22.89222222, -25.78222222,
        -28.24222222]),
 'nfev': 21,
 'fjac': array([[-2.99999999,  0.33333333,  0.33333333,  0.33333333,  0.33333333,
          0.33333333,  0.33333333,  0.33333333,  0.33333333],
        [ 0.        , -0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        , -0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ]]),
 'ipvt': array([1, 2, 3], dtype=int32),
 'qtf': array([2.08951083e-07, 2.05608333e+01, 1.27708333e+01])}

In [10]:
msg

'The relative error between two consecutive iterates is at most 0.000000'

In [11]:
ier

2

That's pretty cryptic and to me at least not helpful. There's no clear fit success/failure flag or message.

**Learning 2: scipy.optimize.curve_fit doesn't help you much to debug failing fits.**

## Tracing the function calls

One thing you can do to undertand what's going on is to trace the optimisation to see the evolution in the parameter estimation.

This is something often done in Bayesian frameworks like [pymc](https://www.pymc.io/welcome.html) or MCMC samplers like [emcee](https://emcee.readthedocs.io/en/stable/) or in neural net fitting with [pytorch](https://pytorch.org/) or [tensorflow](https://www.tensorflow.org/).
But also some nonlinear optimisation packages like [estimagic](https://estimagic.org/) have this built in.

I didn't find a built-in option to do this with scipy or iminuit, and for lmfit I only found a `trace=True` option for [lmfit.conf_interval](https://lmfit.github.io/lmfit-py/confidence.html#confidence-interval-functions) and not the main fit function.

Here we just add print statements printing dicts in [JSONL](https://jsonlines.org/) format, but you could write it to a text or SQLite file or in-memory list if you want to capture it as part of the fit results.


In [12]:
def model(x, b1, b2, b3):
    print({"b1": b1, "b2": b2, "b3": b3})
    return b1 / (1 + np.exp(b2 - b3 * x))

In [13]:
p0=[1, 1, 1]
popt, pcov = scipy.optimize.curve_fit(f=model, xdata=df["x"], ydata=df["y"], p0=p0)

{'b1': 1, 'b2': 1, 'b3': 1}
{'b1': 1.0000000149011612, 'b2': 1.0, 'b3': 1.0}
{'b1': 1.0, 'b2': 1.0000000149011612, 'b3': 1.0}
{'b1': 1.0, 'b2': 1.0, 'b3': 1.0000000149011612}
{'b1': 42.73128723717945, 'b2': -581635.5429887469, 'b3': -75906.38810566434}
{'b1': 42.633884317230134, 'b2': -145840.2885663749, 'b3': -27453.125482927248}
{'b1': 25.6414233988939, 'b2': 19213.87362083769, 'b3': -2163.986665802865}
{'b1': 12.452519069475734, 'b2': 2436.3185266913515, 'b3': -270.42669982149755}
{'b1': 6.246235668851631, 'b2': -986.4341135361835, 'b3': 111.60437183708017}
{'b1': 6.246235761927796, 'b2': -986.4341135361835, 'b3': 111.60437183708017}
{'b1': 6.246235668851631, 'b2': -986.4340988371698, 'b3': 111.60437183708017}
{'b1': 6.246235668851631, 'b2': -986.4341135361835, 'b3': 111.6043735001149}
{'b1': 16.742992104834787, 'b2': -986.4341135361835, 'b3': 111.60437183708017}
{'b1': 16.742992354324812, 'b2': -986.4341135361835, 'b3': 111.60437183708017}
{'b1': 16.742992104834787, 'b2': -986.4340

/Users/cdeil/anaconda3/envs/py-model-fit/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/6v/0_6nt0pj07x9xjhd8qzkyy700000gn/T/ipykernel_20821/188195328.py:2: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = scipy.optimize.curve_fit(f=model, xdata=df["x"], ydata=df["y"], p0=p0)


In [14]:
popt

array([  38.83777778, -986.43411354,  111.60437184])

We see that the optimiser started by evaluating the function at `p0 = [1, 1, 1]` and then took a step `1.0000000149 = 1.49e-8` in each parameter.

Based on that it jumped to completely incorrect parameters where the cost function became `inf`:



In [15]:
b2 = -581635
b3 = -75906
np.exp(b2 - b3 * df["x"][0])

/var/folders/6v/0_6nt0pj07x9xjhd8qzkyy700000gn/T/ipykernel_20821/3346874278.py:3: RuntimeWarning: overflow encountered in exp
  np.exp(b2 - b3 * df["x"][0])


inf

Somehow the optimiser then jumps to `b2` and `b3` values where the `exp` evaluates to `0` and thus the model `y = b1 / (1 + np.exp(b2 - b3 * x))` effectively becomes `y = b1`:

In [16]:
b2 = -986
b3 = 111
np.exp(b2 - b3 * df["x"]).max()

0.0

**Learning 3: Tracing the optimisation can sometimes give insights which parameters and model parts are the problem.**

That said, usually for complex models with many parameters understanding the traces and issues is hard and you'd try other things first.

## iminuit

Let's remove the print statements and tracing from our model and see what iminuit does.

In [17]:
# Do the same failing fit with iminuit and show how it gives a better status and diagnostics with likelihood profile curves
# Show common ways to fix the issue: (a) better starting value  (b) use bounds (c) fix/release parameters (d) grid search followed by migrad

In [18]:
def model(x, b1, b2, b3):
    return b1 / (1 + np.exp(b2 - b3 * x))

## Bad models

* Problem: Highly correlated parameters or fully degenerated parameters
* Solution: Choose model with fewer or different parameters or simply fix some.

In [19]:
# TODO: create example fitting a + bx + c with degenerate a and c

In [20]:
# TODO: show badly parameterised example and how equivalent re-parametrisation makes things simpler

In [21]:
# TODO: add second example of fitting step function location that results in flat minimum area.

def step_model(x, x_step, amplitude):
    return np.where(x > x_step, amplitude, 0)

## Summary and conclusions

We have seen some failing fits and now understand the typical problems. Lessons learned:

* Learning 1: if you get any warnings/errors, don't trust the fit. Fix them.
* Learning 2: scipy.optimize.curve_fit doesn't help you much to debug failing fits.
* Learning 3: Tracing the optimisation can sometimes give insights which parameters and model parts are the problem.
* Learning 4: In practice fixing failing fits usually means finding better initial guesses for the parameters or using bounds.
* Learning 5: Highly correlated or degnerate model parameters are bad. If possible try to avoid them by re-parametrising your model or fixing some parameters to average reasonable values.

Concerning the question if you should use `scipy.optimize` or `iminuit`:

* Learning 7: Both are great, and nowadays `scipy.optimize` has more robust optimizers and supports e.g. setting bounds.
* Learning 8: Some advantages of `iminuit` remain: (a) fix/free parametrs without touching model code, (b) better fit diagnostics and parameter error estimates, (c) easier to use interactively.
